In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Utils.greeks as gr
import Utils.option_pricing_utils as opu

In [81]:
option_exposure = -100000  # Number of options held in portfolio that we want to hedge
S = np.array([49, 48.12, 47.37, 50.25, 51.75, 53.12, 53.0, 51.87, 51.38, 53.0, 49.88, 48.50, 49.88, 50.37, 52.13, 51.88, 52.87, 54.87, 54.62, 55.87, 57.25])
r = 0.05
sigma = 0.20
q = 0
dt = 1/52
T = np.array([20/52-dt*x for x in range(len(S))])
K = 50
type = 'c'

In [82]:
delta_array = gr.get_delta(S, K, T, sigma, r, q, type)
gamma_array = gr.get_gamma(S, K, T, sigma, r, q, type)

c:\Users\Matthew\Documents\Python Scripts\Option_Pricing\Utils\option_pricing_utils.py:7: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (np.log(S/K) + (r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))
c:\Users\Matthew\Documents\Python Scripts\Option_Pricing\Utils\greeks.py:19: RuntimeWarning: invalid value encountered in true_divide
  return norm.pdf(d1)/(S*sigma*np.sqrt(T))


#### Delta Hedging

In [88]:
summary_df = pd.DataFrame(list(zip(S, delta_array, gamma_array)), columns=['Stock Price', 'Delta', 'Gamma'])

In [102]:
summary_df['Shares Held'] = -option_exposure*delta_array
summary_df['Shares Purchased'] = -summary_df['Shares Held'].sub(summary_df['Shares Held'].shift(-1).fillna(0)).shift().fillna(-summary_df['Shares Held'])
summary_df['Cost of Shares Purchased'] = summary_df['Shares Purchased'].mul(summary_df['Stock Price'])
summary_df['Cumulative Cost'] = summary_df['Cost of Shares Purchased'].cumsum()
summary_df['Interest Cost'] = summary_df['Cumulative Cost']*(r*dt)
summary_df['Cumulative Cost Inc. Interest'] =  summary_df['Cumulative Cost'].add(summary_df['Interest Cost'].shift()).fillna(summary_df['Cumulative Cost'])
summary_df['Portfolio Delta'] = option_exposure*summary_df['Delta'] + summary_df['Shares Held']*1
summary_df['Portfolio Gamma'] = option_exposure*summary_df['Gamma'] + summary_df['Shares Held']*0
hedging_cost = -K*option_exposure - summary_df['Cumulative Cost Inc. Interest'].iloc[-1]  # Cash received for stock - total cost  

In [104]:
print(f'Hedging PnL = ${hedging_cost}')
summary_df

Hedging PnL = $-200095.57729936764


,Stock Price,Delta,Gamma,Shares Held,Shares Purchased,Cost of Shares Purchased,Cumulative Cost,Interest Cost,Cumulative Cost Inc. Interest,Portfolio Delta,Portfolio Gamma
0,49.00,0.521605,0.065544,52160.466107,52160.466107,2.555863e+06,2.555863e+06,2457.560422,2.555863e+06,0.0,-6554.403935
1,48.12,0.458010,0.068197,45800.967378,-6359.498729,-3.060191e+05,2.249844e+06,2163.311308,2.252301e+06,0.0,-6819.691750
2,47.37,0.400027,0.069313,40002.677068,-5798.290310,-2.746650e+05,1.975179e+06,1899.210335,1.977342e+06,0.0,-6931.259923
3,50.25,0.596282,0.067394,59628.194505,19625.517437,9.861823e+05,2.961361e+06,2847.462500,2.963260e+06,0.0,-6739.399626
4,51.75,0.692952,0.061193,69295.199497,9667.004992,5.002675e+05,3.461629e+06,3328.488950,3.464476e+06,0.0,-6119.289487
5,53.12,0.773821,0.052717,77382.062237,8086.862740,4.295741e+05,3.891203e+06,3741.541016,3.894531e+06,0.0,-5271.660409
6,53.00,0.771289,0.055034,77128.922763,-253.139473,-1.341639e+04,3.877786e+06,3728.640639,3.881528e+06,0.0,-5503.435526
7,51.87,0.706151,0.066399,70615.130773,-6513.791990,-3.378704e+05,3.539916e+06,3403.765264,3.543645e+06,0.0,-6639.912877
8,51.38,0.674190,0.072984,67418.957069,-3196.173704,-1.642194e+05,3.375696e+06,3245.861990,3.379100e+06,0.0,-7298.418936
9,53.00,0.786526,0.059685,78652.638048,11233.680979,5.953851e+05,3.971082e+06,3818.347655,3.974327e+06,0.0,-5968.497499
